## 1. Imports

In [ ]:
import random
import math
import copy

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor

import os
import tensorflow as tf
# import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model

import torch
from kan import KAN

from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#%load_ext autotime
#%load_ext autotime

In [ ]:
#from tools import KAN_es
from algos import vector_pred_skl, vector_pred_NN, vector_pred_KAN, multi_exp

## 2. Data preparation

### 2.1. Loading data

In [ ]:
'''
initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_vld.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_vld.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_vld.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
'''
# !Some troubles with *.csv naming!

initial_trn_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_trn.csv")
initial_vld_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_tst.csv")
initial_tst_data_fdfs = pd.read_csv("datasets_mtgm\mtgm_fdfs_10k_pro.csv")

initial_trn_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_trn.csv")
initial_vld_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_tst.csv")
initial_tst_data_udfs = pd.read_csv("datasets_mtgm\mtgm_udfs_10k_pro.csv")

initial_trn_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_trn.csv")
initial_vld_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_tst.csv")
initial_tst_data_udus = pd.read_csv("datasets_mtgm\mtgm_udus_10k_pro.csv")

In [ ]:
data_col_names = initial_trn_data_fdfs.columns

### 2.2. Scaling data

In [ ]:
mm_scaler = MinMaxScaler()
mm_scaler.fit(pd.concat([initial_trn_data_fdfs,
                         initial_vld_data_fdfs],
                        axis=0, sort=False, ignore_index=True))

In [ ]:
'''
scaled_trn_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
scaled_vld_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
scaled_tst_data_fdfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)

scaled_trn_data_udfs = pd.DataFrame(mm_scaler.transform(initial_trn_data_udfs),columns=data_col_names)
scaled_vld_data_udfs = pd.DataFrame(mm_scaler.transform(initial_vld_data_udfs),columns=data_col_names)
scaled_tst_data_udfs = pd.DataFrame(mm_scaler.transform(initial_tst_data_udfs),columns=data_col_names)

scaled_trn_data_udus = pd.DataFrame(mm_scaler.transform(initial_trn_data_udus),columns=data_col_names)
scaled_vld_data_udus = pd.DataFrame(mm_scaler.transform(initial_vld_data_udus),columns=data_col_names)
scaled_tst_data_udus = pd.DataFrame(mm_scaler.transform(initial_tst_data_udus),columns=data_col_names)
'''
trn = pd.DataFrame(mm_scaler.transform(initial_trn_data_fdfs),columns=data_col_names) 
vld = pd.DataFrame(mm_scaler.transform(initial_vld_data_fdfs),columns=data_col_names)
tst = pd.DataFrame(mm_scaler.transform(initial_tst_data_fdfs),columns=data_col_names)


for SAD:

In [ ]:
'''
pd.concat([
pd.concat([ trn.iloc[:, :124], trn.loc[:,'H1_8'], trn.loc[:,'H2_8'], trn.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False),
pd.concat([ vld.iloc[:, :124], vld.loc[:,'H1_8'], vld.loc[:,'H2_8'], vld.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)],
axis=0, sort=False, ignore_index=False)#.to_csv('trn_GMT.csv')

pd.concat([
pd.concat([ trn.iloc[:, :31], trn.loc[:,'H1_8'], trn.loc[:,'H2_8'], trn.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False),
pd.concat([ vld.iloc[:, :31], vld.loc[:,'H1_8'], vld.loc[:,'H2_8'], vld.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)],
axis=0, sort=False, ignore_index=False)#.to_csv('trn_G.csv')

pd.concat([
pd.concat([ trn.iloc[:, 31:62], trn.loc[:,'H1_8'], trn.loc[:,'H2_8'], trn.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False),
pd.concat([ vld.iloc[:, 31:62], vld.loc[:,'H1_8'], vld.loc[:,'H2_8'], vld.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)],
axis=0, sort=False, ignore_index=False)#.to_csv('trn_M.csv')

pd.concat([
pd.concat([ trn.iloc[:, 62:124], trn.loc[:,'H1_8'], trn.loc[:,'H2_8'], trn.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False),
pd.concat([ vld.iloc[:, 62:124], vld.loc[:,'H1_8'], vld.loc[:,'H2_8'], vld.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)],
axis=0, sort=False, ignore_index=False)#.to_csv('trn_T.csv')

pd.concat([ tst.iloc[:, :124], tst.loc[:,'H1_8'], tst.loc[:,'H2_8'], tst.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)#.to_csv('tst_GMT.csv')

pd.concat([ tst.iloc[:, :31], tst.loc[:,'H1_8'], tst.loc[:,'H2_8'], tst.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)#.to_csv('tst_G.csv')

pd.concat([ tst.iloc[:, 31:62], tst.loc[:,'H1_8'], tst.loc[:,'H2_8'], tst.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)#.to_csv('tst_M.csv')

pd.concat([ tst.iloc[:, 62:124], tst.loc[:,'H1_8'], tst.loc[:,'H2_8'], tst.loc[:,'H3_8'] ],
          axis=1, sort=False, ignore_index=False)#.to_csv('tst_T.csv')
'''

# 3. Experiment №2
* Input data: G, M, T, GMT.
* Models: Group Method Data Holding, Random Forest, Gradient Boosting, Multi-Layer Perceptron, Kolmogorov-Arnold Network.
* Single mode predicting: seperate model per each predicting depth.

In [ ]:
NUM_ITER = 25

from sklearn.base import BaseEstimator
from gmdh import Ria

def skl_regr_wrap(class_model, default_init_kwargs = {}, default_fit_kwargs = {}):
    '''
    Wrapper for regressor classes with fit, predict methods. Makes regressor compatable with skl cross-validation.
    Returns new class.
    '''
    class MyEstimator(BaseEstimator):
        '''
        Provides Sci-kit learn compatable Regressor class from class model with fit, predict functions 
        '''
        def __init__(self, *, init_kwargs=default_init_kwargs, fit_kwargs=default_fit_kwargs, random_state=1):
            self.init_kwargs = init_kwargs
            self.fit_kwargs = fit_kwargs
            self.class_model = class_model

        def fit(self, X, y):
            self.model = class_model(**self.init_kwargs)
            self.model.fit(X=np.array(X), y=np.array(y), **self.fit_kwargs)
            self.is_fitted_ = True
            return self

        def predict(self, X):
            return self.model.predict(X=X)
        
        
    return MyEstimator

### 3.1 GMDH

In [ ]:
num_iter=NUM_ITER
alg_name_to_save = 'GMDH'


l_algos_names=['G_GMDH', 'M_GMDH', 'T_GMDH', 'GMT_GMDH']

l_algos=[vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4)]

l_geophysical_method = ['G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4)]

l_kwargs=[{'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True},
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True},
          ]

l_metrics_names=['rmse_H1_8', 'rmse_H2_8', 'rmse_H3_8', 
                 'mae_H1_8', 'mae_H2_8', 'mae_H3_8', 
                 'mape_H1_8', 'mape_H2_8', 'mape_H3_8', 
                 'r2_H1_8', 'r2_H2_8', 'r2_H3_8']


full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

In [ ]:
full_df.to_excel(f'full_metrics_2_{alg_name_to_save}.xlsx')
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel(f'aggr_metrics_2_{alg_name_to_save}.xlsx')
aggr_df

### 3.2 RF

In [ ]:
num_iter=NUM_ITER
alg_name_to_save = 'RF'


l_algos_names=['G_RF', 'M_RF', 'T_RF', 'GMT_RF']

l_algos=[vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4)]

l_geophysical_method = ['G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4)]

l_kwargs=[{'class_model': RandomForestRegressor, 'multioutput': True}, {'class_model': RandomForestRegressor, 'multioutput': True}, {'class_model': RandomForestRegressor, 'multioutput': True}, {'class_model': RandomForestRegressor, 'multioutput': True}, 
          ]

l_metrics_names=['rmse_H1_8', 'rmse_H2_8', 'rmse_H3_8', 
                 'mae_H1_8', 'mae_H2_8', 'mae_H3_8', 
                 'mape_H1_8', 'mape_H2_8', 'mape_H3_8', 
                 'r2_H1_8', 'r2_H2_8', 'r2_H3_8']


full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

In [ ]:
full_df.to_excel(f'full_metrics_2_{alg_name_to_save}.xlsx')
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel(f'aggr_metrics_2_{alg_name_to_save}.xlsx')
aggr_df

### 3.3 GB

In [ ]:
num_iter=NUM_ITER
alg_name_to_save = 'GB'


l_algos_names=['G_GB', 'M_GB', 'T_GB', 'GMT_GB']

l_algos=[vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4)]

l_geophysical_method = ['G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4)]

l_kwargs=[{'class_model': GradientBoostingRegressor, 'multioutput': True}, {'class_model': GradientBoostingRegressor, 'multioutput': True}, {'class_model': GradientBoostingRegressor, 'multioutput': True}, {'class_model': GradientBoostingRegressor, 'multioutput': True}, 
]

l_metrics_names=['rmse_H1_8', 'rmse_H2_8', 'rmse_H3_8', 
                 'mae_H1_8', 'mae_H2_8', 'mae_H3_8', 
                 'mape_H1_8', 'mape_H2_8', 'mape_H3_8', 
                 'r2_H1_8', 'r2_H2_8', 'r2_H3_8']


full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

In [ ]:
full_df.to_excel(f'full_metrics_2_{alg_name_to_save}.xlsx')
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel(f'aggr_metrics_2_{alg_name_to_save}.xlsx')
aggr_df

### 3.4 MLP keras

In [ ]:
num_iter=NUM_ITER
alg_name_to_save = 'MLP'


l_algos_names=['G_MLP', 'M_MLP', 'T_MLP', 'GMT_MLP']

l_algos=[vector_pred_NN, vector_pred_NN, vector_pred_NN, vector_pred_NN,]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4)]

l_geophysical_method = ['G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4)]

l_kwargs=[{'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},
          {'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},
          {'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},
          {'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},]

l_metrics_names=['rmse_H1_8', 'rmse_H2_8', 'rmse_H3_8', 
                 'mae_H1_8', 'mae_H2_8', 'mae_H3_8', 
                 'mape_H1_8', 'mape_H2_8', 'mape_H3_8', 
                 'r2_H1_8', 'r2_H2_8', 'r2_H3_8']


full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

In [ ]:
full_df.to_excel(f'full_metrics_2_{alg_name_to_save}.xlsx')
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel(f'aggr_metrics_2_{alg_name_to_save}.xlsx')
aggr_df

### 3.5 KAN

In [ ]:
num_iter=NUM_ITER
alg_name_to_save = 'KAN'


l_algos_names=['G_KAN', 'M_KAN', 'T_KAN', 'GMT_KAN']

l_algos=[vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4)]

l_geophysical_method = ['G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4)]

l_kwargs=[{'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          ]

l_metrics_names=['rmse_H1_8', 'rmse_H2_8', 'rmse_H3_8', 
                 'mae_H1_8', 'mae_H2_8', 'mae_H3_8', 
                 'mape_H1_8', 'mape_H2_8', 'mape_H3_8', 
                 'r2_H1_8', 'r2_H2_8', 'r2_H3_8']


full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

In [ ]:
full_df.to_excel(f'full_metrics_2_{alg_name_to_save}.xlsx')
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel(f'aggr_metrics_2_{alg_name_to_save}.xlsx')
aggr_df

### 4.6 Extra: MLP from skl

In [ ]:
num_iter=NUM_ITER
alg_name_to_save = 'MLP_skl'


l_algos_names=['G_MLP_skl', 'M_MLP_skl', 'T_MLP_skl', 'GMT_MLP_skl']

l_algos=[vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4)]

l_geophysical_method = ['G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4)]

l_kwargs=[{'class_model': MLPRegressor, 'model_kwargs': dict(hidden_layer_sizes=[32], activation='logistic', validation_fraction=2/7, learning_rate_init=0.001, early_stopping=True, tol=0.001, n_iter_no_change=500, max_iter=50000, max_fun=50000), 'multioutput': True}, 
          {'class_model': MLPRegressor, 'model_kwargs': dict(hidden_layer_sizes=[32], activation='logistic', validation_fraction=2/7, learning_rate_init=0.001, early_stopping=True, tol=0.001, n_iter_no_change=500, max_iter=50000, max_fun=50000), 'multioutput': True}, 
          {'class_model': MLPRegressor, 'model_kwargs': dict(hidden_layer_sizes=[32], activation='logistic', validation_fraction=2/7, learning_rate_init=0.001, early_stopping=True, tol=0.001, n_iter_no_change=500, max_iter=50000, max_fun=50000), 'multioutput': True}, 
          {'class_model': MLPRegressor, 'model_kwargs': dict(hidden_layer_sizes=[32], activation='logistic', validation_fraction=2/7, learning_rate_init=0.001, early_stopping=True, tol=0.001, n_iter_no_change=500, max_iter=50000, max_fun=50000), 'multioutput': True}, 
]

l_metrics_names=['rmse_H1_8', 'rmse_H2_8', 'rmse_H3_8', 
                 'mae_H1_8', 'mae_H2_8', 'mae_H3_8', 
                 'mape_H1_8', 'mape_H2_8', 'mape_H3_8', 
                 'r2_H1_8', 'r2_H2_8', 'r2_H3_8']


full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

In [ ]:
full_df.to_excel(f'full_metrics_2_{alg_name_to_save}.xlsx')
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel(f'aggr_metrics_2_{alg_name_to_save}.xlsx')
aggr_df

# 4. All experiments in 1 launch:

In [ ]:
l_algos_names=['G_GMDH', 'M_GMDH', 'T_GMDH', 'GMT_GMDH',
               'G_RF', 'M_RF', 'T_RF', 'GMT_RF',
               'G_GB', 'M_GB', 'T_GB', 'GMT_GB',
               'G_MLP', 'M_MLP', 'T_MLP', 'GMT_MLP',
               'G_KAN', 'M_KAN', 'T_KAN', 'GMT_KAN']

l_algos=[vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl, 
         vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl, 
         vector_pred_skl, vector_pred_skl, vector_pred_skl, vector_pred_skl, 
         vector_pred_NN, vector_pred_NN, vector_pred_NN, vector_pred_NN, 
         vector_pred_KAN, vector_pred_KAN, vector_pred_KAN, vector_pred_KAN]

Data = (trn, vld, tst)

mult_data = [Data for i in range(4*5)]

l_geophysical_method = ['G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT',
                        'G', 'M', 'T', 'GMT']

output_parameter = ['H1_8', 'H2_8', 'H3_8']
l_output_parameter = [output_parameter for i in range(4*5)]

l_kwargs=[{'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True}, 
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True},
          {'class_model': skl_regr_wrap(class_model=Ria), 'model_kwargs': dict(fit_kwargs = dict(k_best=15, p_average=1)), 'multioutput': True},
          
          {'class_model': RandomForestRegressor, 'multioutput': True}, {'class_model': RandomForestRegressor, 'multioutput': True}, {'class_model': RandomForestRegressor, 'multioutput': True}, {'class_model': RandomForestRegressor, 'multioutput': True}, 
          
          {'class_model': GradientBoostingRegressor, 'multioutput': True}, {'class_model': GradientBoostingRegressor, 'multioutput': True}, {'class_model': GradientBoostingRegressor, 'multioutput': True}, {'class_model': GradientBoostingRegressor, 'multioutput': True}, 

          {'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},
          {'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},
          {'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},
          {'learning_rate': 0.001, 'tol':0.001, 'n_iter_no_change':500, 'max_epochs':50000, 'rel_batch_size':0.05, 'multioutput': True},
          
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          {'K':3, 'hidden_neurons':1, 'learning_rate':0.1, 'tol':0.001, 'n_iter_no_change':25, 'max_epochs':500, 'lamb':0, 'multioutput': True},
          ]

l_metrics_names=['rmse_H1_8', 'rmse_H2_8', 'rmse_H3_8', 
                 'mae_H1_8', 'mae_H2_8', 'mae_H3_8', 
                 'mape_H1_8', 'mape_H2_8', 'mape_H3_8', 
                 'r2_H1_8', 'r2_H2_8', 'r2_H3_8']

num_iter=NUM_ITER

In [ ]:
full_df = multi_exp(l_algos_names=l_algos_names,
                    l_algos=l_algos,
                    mult_data=mult_data,
                    l_geophysical_method=l_geophysical_method,
                    l_output_parameter=l_output_parameter,
                    l_kwargs=l_kwargs,
                    l_metrics_names=l_metrics_names,
                    num_iter=num_iter)

In [ ]:
full_df

In [ ]:
full_df.to_excel('full_metrics_2.xlsx')
#pd.read_excel('full_metrics.xlsx').drop('Unnamed: 0', axis=1)

In [ ]:
aggr_df = full_df.groupby(['alg_name']).agg(["mean", "std"]).drop(['iter'], axis=1)
aggr_df.to_excel('aggr_metrics_2.xlsx')
aggr_df